In [2]:
import numpy as np
import sys
sys.path.insert(1,'../../src')
from _generate_time_series import *
from _shear_frame import *
from utils_OB import *
from _clustering import *
import _tracking as tk
from utils_JK import *
import seaborn as sns
sns.set_theme(context='paper',font_scale=1.0, style = 'ticks', color_codes='deep')

%load_ext autoreload
%autoreload 2

In [3]:
data_true = np.load('./test_results_flipped/test_3_true_flipped.npz',allow_pickle=True)
true_f = data_true['arr_0']
true_phi = data_true['arr_1']

In [4]:
n_test = np.array([20,25,30,35,40])
m_test = np.array([5,8,11,14,17,20,26,32])
d_track_threshold_test = np.array([0.01, 0.05, 0.075 , 0.10, 0.20, 0.30, 0.35, 0.50, 0.80])

In [12]:
for run_i in range(10):
    print('run: {}'.format(run_i))

    datai = np.load('./test_results_flipped/test_3_run_'+str(run_i)+'_flipped.npz',allow_pickle=True)
    freqs_outi = datai['arr_0']
    phis_outi = datai['arr_1']

    result_dict_average = {}
    result_dict_linear_extrapolation = {}

    for method in ['average', 'linear_extrapolation']:    
        for ni in n_test:
            for mi in m_test:
                for d_track_threshold_i in d_track_threshold_test:
                    mode_traces_i = tk.perform_tracking(freqs_outi, phis_outi, n = ni, m = mi, d_track_threshold=d_track_threshold_i,method=method, split=True)
                    if len(mode_traces_i) != 4:
                        if method == 'average':
                            result_dict_average[ni, mi, d_track_threshold_i] = None
                        else:
                            result_dict_linear_extrapolation[ni, mi, d_track_threshold_i] = None
                    else:
                        temp_n_skipped_time_seg = []
                        temp_time_seg = []
                        temp_frequencies = []
                        mode_traces_i = tk.sort_traces(mode_traces_i, true_f[ni], true_phi[:,:,ni])
                        for k, mode_traces_ik in enumerate(mode_traces_i):
                            mode_traces_ik.count_missed_time_segments(freqs_outi[ni:],phis_outi[ni:],true_mode_f=true_f[ni:,k],true_mode_phi=true_phi[k,:,ni:],lim=d_track_threshold_i, n = ni)
                            temp_n_skipped_time_seg.append(mode_traces_ik.n_skipped_time_seg + mode_traces_ik.n_false_positives) 
                            temp_time_seg.append(mode_traces_ik.time_seg)
                            temp_frequencies.append(mode_traces_ik.frequencies)
                        if method == 'average':
                            result_dict_average[ni, mi, d_track_threshold_i] = temp_time_seg, temp_frequencies, temp_n_skipped_time_seg
                        else:
                            result_dict_linear_extrapolation[ni, mi, d_track_threshold_i] = temp_time_seg, temp_frequencies, temp_n_skipped_time_seg

    np.savez('./result_first_algorithm_flipped/result_average '+str(run_i), result_dict=result_dict_average)
    np.savez('./result_first_algorithm_flipped/result_linear_extrapolation '+str(run_i), result_dict=result_dict_linear_extrapolation)

run: 0
run: 1
run: 2
run: 3
run: 4
run: 5
run: 6
run: 7
run: 8
run: 9
